In [84]:
import pandas as pd
import numpy as np
from numpy.lib.stride_tricks import sliding_window_view
from collections import Counter

In [5]:
df = pd.read_csv('airquality.csv', dtype={'station_id': str})
station_ids = df['station_id'].unique()
station_ids

array(['001001', '001002', '001003', '001004', '001005', '001006',
       '001007', '001008', '001009', '001010', '001011', '001012',
       '001013', '001014', '001015', '001016', '001017', '001018',
       '001019', '001020', '001021', '001022', '001023', '001024',
       '001025', '001026', '001027', '001028', '001029', '001030',
       '001031', '001032', '001033', '001034', '001035', '001036',
       '004002', '004003', '004007', '004008', '004009', '004011',
       '004014', '004017', '004018', '004019', '004020', '006001',
       '006002', '006003', '006004', '006005', '006006', '006007',
       '006008', '006010', '006011', '006012', '006013', '006014',
       '006015', '006016', '006017', '006019', '006020', '006021',
       '006022', '006023', '006024', '006025', '006026', '006027',
       '006028', '006040', '009016', '009017', '009018', '009019',
       '009020', '009021', '009022', '009023', '009024', '009025',
       '009026', '009027', '009028', '009029', '009030', '0090

In [12]:
location_df = pd.read_csv('station.csv', dtype={'station_id': str})
location_df = location_df.drop(columns=['name_chinese', 'name_english', 'district_id'])
location_ids = location_df['station_id'].unique()
location_ids

array(['001001', '001002', '001003', '001004', '001005', '001006',
       '001007', '001008', '001009', '001010', '001011', '001012',
       '001013', '001014', '001015', '001016', '001017', '001018',
       '001019', '001020', '001021', '001022', '001023', '001024',
       '001025', '001026', '001027', '001028', '001029', '001030',
       '001031', '001032', '001033', '001034', '001035', '001036',
       '004002', '004003', '004007', '004008', '004009', '004011',
       '004014', '004017', '004018', '004019', '004020', '006001',
       '006002', '006003', '006004', '006005', '006006', '006007',
       '006008', '006010', '006011', '006012', '006013', '006014',
       '006015', '006016', '006017', '006019', '006020', '006021',
       '006022', '006023', '006024', '006025', '006026', '006027',
       '006028', '006040', '009016', '009017', '009018', '009019',
       '009020', '009021', '009022', '009023', '009024', '009025',
       '009026', '009027', '009028', '009029', '009030', '0090

In [17]:
df = pd.merge(df, location_df, how='left', on='station_id')
df

,station_id,time,PM25_Concentration,PM10_Concentration,NO2_Concentration,CO_Concentration,O3_Concentration,SO2_Concentration,latitude,longitude
0,001001,2014-05-01 00:00:00,138.0,159.4,56.3,0.900,50.8,17.2,40.090679,116.173553
1,001001,2014-05-01 01:00:00,124.0,163.9,38.7,0.900,51.1,17.9,40.090679,116.173553
2,001001,2014-05-01 02:00:00,127.0,148.4,55.6,1.000,27.2,16.6,40.090679,116.173553
3,001001,2014-05-01 03:00:00,129.0,145.6,65.7,1.000,9.7,16.7,40.090679,116.173553
4,001001,2014-05-01 04:00:00,119.0,119.3,66.9,1.000,2.0,16.5,40.090679,116.173553
...,...,...,...,...,...,...,...,...,...,...
2891388,372002,2015-04-30 18:00:00,54.0,90.0,16.0,1.326,84.0,22.0,24.417777,111.526388
2891389,372002,2015-04-30 19:00:00,64.0,83.0,12.0,1.099,68.0,21.0,24.417777,111.526388
2891390,372002,2015-04-30 20:00:00,61.0,81.0,29.0,1.324,39.0,21.0,24.417777,111.526388
2891391,372002,2015-04-30 21:00:00,56.0,84.0,19.0,1.142,26.0,21.0,24.417777,111.526388


In [55]:
longest = 0
longest_subdf = None
for station_id, subdf in df.groupby('station_id'):
    subdf = subdf.drop(columns=['station_id', 'longitude', 'latitude'])
    subdf = subdf.set_index('time')
    subdf.sort_index(inplace=True)
    dates = subdf.index.str[:-9].unique()
    if len(dates) > longest:
        longest = len(dates)
        longest_station = station_id
        longest_subdf = subdf

dates = longest_subdf.index.str[:-9].unique()
date_to_int = {date: i for i, date in enumerate(dates)}
int_to_date = {i: date for date, i in date_to_int.items()}

hour_to_int = {f'{i:02d}:00:00': i for i in range(24)}
int_to_hour = {i: hour for hour, i in hour_to_int.items()}

station_id_to_int = {sid: i for i, sid in enumerate(station_ids)}
int_to_station_id = {i: sid for sid, i in station_id_to_int.items()}

In [53]:
data = np.full((len(station_ids), 365, 24, 6), -1, dtype=np.float32)
for station_id, subdf in df.groupby('station_id'):
    subdf = subdf.drop(columns=['station_id', 'longitude', 'latitude'])
    subdf = subdf.set_index('time')
    subdf.sort_index(inplace=True)
    for time in subdf.index:
        date = time[:-9]
        hour = time[-8:]
        if date in date_to_int and hour in hour_to_int:
            date_idx = date_to_int[date]
            hour_idx = hour_to_int[hour]
            station_idx = station_id_to_int[station_id]
            data[station_idx, date_idx, hour_idx, 0] = subdf.at[time, 'PM25_Concentration']
            data[station_idx, date_idx, hour_idx, 1] = subdf.at[time, 'PM10_Concentration']
            data[station_idx, date_idx, hour_idx, 2] = subdf.at[time, 'NO2_Concentration']
            data[station_idx, date_idx, hour_idx, 3] = subdf.at[time, 'CO_Concentration']
            data[station_idx, date_idx, hour_idx, 4] = subdf.at[time, 'O3_Concentration']
            data[station_idx, date_idx, hour_idx, 5] = subdf.at[time, 'SO2_Concentration']
data

array([[[[ 1.380e+02,  1.594e+02,  5.630e+01,  9.000e-01,  5.080e+01,
           1.720e+01],
         [ 1.240e+02,  1.639e+02,  3.870e+01,  9.000e-01,  5.110e+01,
           1.790e+01],
         [ 1.270e+02,  1.484e+02,  5.560e+01,  1.000e+00,  2.720e+01,
           1.660e+01],
         ...,
         [ 1.190e+02,  2.804e+02,  6.300e+00,  4.000e-01,  8.560e+01,
           1.130e+01],
         [ 4.800e+01,  6.100e+01,  4.100e+00,  3.000e-01,  8.360e+01,
           9.900e+00],
         [ 3.700e+01,  3.890e+01,  4.000e+00,  3.000e-01,  8.250e+01,
           8.900e+00]],

        [[ 4.900e+01,  7.010e+01,  6.300e+00,  3.000e-01,  7.860e+01,
           8.900e+00],
         [ 4.200e+01,  5.560e+01,  8.800e+00,  4.000e-01,  7.480e+01,
           9.800e+00],
         [ 3.400e+01,        nan,  6.900e+00,  4.000e-01,  7.480e+01,
           9.900e+00],
         ...,
         [ 8.000e+00,  1.420e+01,  1.510e+01,  2.000e-01,  8.620e+01,
           6.500e+00],
         [ 3.300e+01,  5.470e+01,  3.010

In [51]:
null_day_count = np.sum(np.all(data == -1, axis=(2, 3)))
null_day_percent = null_day_count / (len(station_ids) * 365)
print(f'Number of days with all values -1: {null_day_count}')
print(f'Proportion of days with all values -1: {null_day_percent:.3f}')

num_nulls = np.sum(data == -1)
non_null_day_nulls = num_nulls - null_day_count * 24 * 6
non_null_day_proportion = non_null_day_nulls / ((len(station_ids) * 365 - null_day_count) * 24 * 6)
print(f'Number of all -1 values in data excluding null days: {non_null_day_nulls}')
print(f'Proportion of all -1 values in non-null days: {non_null_day_proportion:.3f}')


Number of days with all values -1: 24048
Proportion of days with all values -1: 0.151
Number of all -1 values in data excluding null days: 2157486
Proportion of all -1 values in non-null days: 0.111


In [70]:
null_day_indices = np.where(np.all(data == -1, axis=(2, 3)))

station_idxs, date_idxs = null_day_indices
null_days_set = {
    (int_to_station_id[s], int_to_date[d])
    for s, d in zip(station_idxs, date_idxs)
}
null_days_list = [
    (int_to_station_id[s], int_to_date[d])
    for s, d in zip(station_idxs, date_idxs)
]

date_counter = Counter(d for _, d in null_days_list)
max_date, max_count = date_counter.most_common(1)[0]
null_station_proportion = max_count / len(station_ids)

print(f"Date with most null stations: {max_date} ({max_count} stations)")
print(f"Proportion of stations with null values on {max_date}: {null_station_proportion:.3f}")

Date with most null stations: 2014-05-07 (259 stations)
Proportion of stations with null values on 2014-05-07: 0.593


In [89]:
data_flat = data.reshape(len(station_ids), 365*24, 6)
data_windowed = sliding_window_view(data_flat, window_shape=24, axis=1)
data_windowed = data_windowed.transpose(1, 0, 3, 2)
data_windowed.shape

(8737, 437, 24, 6)

In [ ]:
np.save('data_windowed.npy', data_windowed)